# Trabalho Prático 01 - Computação Gráfica SCC0650

##### Luiz Fernando Rabelo (11796893) e Matheus Bermudes Viana (11849797)

### Importação de Bibliotecas

No primeiro passo, importamos as bibliotecas padrão vistas em aula (OpenGL, _glfw_ para gerenciamento das janelas e _numpy_ para operações matemáticas e para manipulação de vetores e matrizes).

In [1351]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicialização da Janela GLFW

Inicializamos a janela com uma largura de 1000 e altura de 800, com o título "Trabalho Prático 1", determinando que inicialmente ela não será exibida. Optamos por passar parâmetros nulos para _monitor_ e _share_.

In [1352]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(1000, 800, 'Trabalho Prático 1', None, None)
glfw.make_context_current(window)

### Definição de Código GLSL para Vertex Shader e Fragment Shader

A fim de possibilitarmos as transformações e configurações de cores, atribuímos códigos GLSL para `vertex_code` e `fragment_code`:

In [1353]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisição de slot para a GPU para Vertex e Fragment Shaders

In [1354]:
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associação do Código Fonte aos Slots Solicitados

In [1355]:
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilação do Vertex e Fragment Shaders

In [1356]:
# Compilação do Vertex Shader:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

# Compilação do Fragment Shader:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Linkagem do Programa

In [1357]:
# Associação dos Shaders complilados ao programa principal:
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Construção do programa:
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Declaração do programa atual como o default:
glUseProgram(program)

### Preparação dos Dados que Serão Enviados para a GPU

A fim de modularizar a criação das coordenadas, foram desenvolvidas algumas funções responsáveis por construir os vértices dos objetos. Cada uma dessas funções retorna uma lista de tuplas representando as coordenadas computadas do objeto.

##### Função para Criação dos Vértices de um Círculo Genérico

In [1358]:
def create_circle_vertices(radius, total_vertices):
    vertices = []

    step = 2 * np.pi / total_vertices
    angle = 0

    for i in range(total_vertices):
        vertices.append((radius * np.cos(angle), radius * np.sin(angle)))
        angle += step

    return vertices

##### Função para Criação dos Vértices da Grama

In [1359]:
def create_grass_vertices():
    vertices = []

    for i in range(-13, 11, 1):
        vertices.append((i/10, -0.9 if i % 3 == 0 else -1))

    for i in range(-13, 11, 1):
        vertices.append((i/10 - 0.165, -0.9 if i % 3 == 0 else -1))

    return vertices

##### Função para Criação dos Vértices do Coração

In [1360]:
def create_heart_vertices():
    vertices = []

    vertices.append([0.20,0.24])
    vertices.append([0.19,0.25])
    vertices.append([0.18,0.25])
    vertices.append([0.17,0.25])
    vertices.append([0.16,0.24])
    vertices.append([0.15,0.23])
    vertices.append([0.15,0.22])
    vertices.append([0.15,0.21])
    vertices.append([0.15,0.20])
    vertices.append([0.16,0.19])
    vertices.append([0.17,0.18])
    vertices.append([0.18,0.17])
    vertices.append([0.19,0.16])
    vertices.append([0.20,0.15])
    vertices.append([0.21,0.16])
    vertices.append([0.22,0.17])
    vertices.append([0.23,0.18])
    vertices.append([0.24,0.19])
    vertices.append([0.25,0.20])
    vertices.append([0.25,0.21])
    vertices.append([0.25,0.22])
    vertices.append([0.25,0.23])
    vertices.append([0.24,0.24])
    vertices.append([0.23,0.25])
    vertices.append([0.22,0.25])
    vertices.append([0.21,0.25])

    return vertices

##### Função para Criação dos Vértices do Corpo de uma Pessoa

In [1361]:
def create_person_body_vertices():
    vertices = []

    vertices.append([0.0,0.0])
    vertices.append([0.0,-0.15])
    vertices.append([0.1,-0.2])
    vertices.append([0.0,-0.15])
    vertices.append([-0.1,-0.2])
    vertices.append([0.0,-0.15])
    vertices.append([0.0,-0.3])
    vertices.append([-0.05,-0.45])
    vertices.append([0.0,-0.3])
    vertices.append([0.05,-0.45])
    vertices.append([0.0,-0.3])

    return vertices

##### Função para Criação dos Vértices do Cabo de um Catavento

In [1362]:
def create_wind_mill_handle_vertices(offset_x, offset_y):
    vertices = []

    vertices.append([+0.002 + offset_x,  0.0 + offset_y])
    vertices.append([-0.002 + offset_x,  0.0 + offset_y])
    vertices.append([-0.002 + offset_x, -0.3 + offset_y])
    vertices.append([+0.002 + offset_x, -0.3 + offset_y])

    return vertices

##### Função para Criação dos Vértices das Pás de um Catavento

In [1363]:
def create_wind_mill_shovel_vertices(offset_x, offset_y):
    vertices = []
    
    step = np.pi / 3.5
    angle = 0.0
    radius = 0.1
    
    for i in range(12):
        small_step = step / 7
        vertices.append([0.0 + offset_x, 0.0 + offset_y])
        vertices.append([0.7 * radius * np.cos(angle + 2 * small_step) + offset_x,
                         0.7 * radius * np.sin(angle + 2 * small_step) + offset_y])
        vertices.append([radius * np.cos(angle + 4 * small_step) + offset_x,
                         radius * np.sin(angle + 4 * small_step) + offset_y])
        vertices.append([0.7 * radius * np.cos(angle + 6 * small_step) + offset_x,
                         0.7 * radius * np.sin(angle + 6 * small_step) + offset_y])   
        angle += step

    return vertices

##### Função para Criação dos Vértices do Tronco da Árvore

In [1364]:
def create_tree_trunk_vertices(offset_x, offset_y):
    vertices = []

    vertices.append((0 + offset_x, 0 + offset_y))
    vertices.append((.1 + offset_x, -.4 + offset_y))
    vertices.append((-.1 + offset_x, -.4 + offset_y))

    return vertices

##### Função para Criação dos Vértices da Copa da Árvore

In [1365]:
def create_tree_top_vertices(offset_x, offset_y):
    vertices = []

    angle = 0
    horizontal_wave = []

    for i in range(31):
        horizontal_wave.append([offset_x + 0.1 * np.cos(angle), offset_y + 0.1 * np.sin(angle)])
        angle += np.pi / 30
    
    for vertex in horizontal_wave:
        vertices.append([vertex[0], vertex[1]])

    for vertex in horizontal_wave:
        vertices.append([vertex[0] + 0.2, vertex[1]])

    for vertex in horizontal_wave:
        vertices.append([vertex[0], vertex[1] - 0.1])

    for vertex in horizontal_wave:
        vertices.append([vertex[0] + 0.2, vertex[1] - 0.1])

    return vertices


##### Criação dos Vértices dos Objetos

In [1366]:
# Criação dos vértices do sol:
sun_start = 0
sun_vertices = create_circle_vertices(0.2, 10)
sun_end = sun_start + len(sun_vertices)

# Criação dos vértices da lua:
moon_start = sun_end
moon_vertices = create_circle_vertices(0.15, 30)
moon_end = moon_start + len(moon_vertices)

# Criação dos vértices da grama:
grass_start = moon_end
grass_vertices = create_grass_vertices()
grass_end = grass_start + len(grass_vertices)

# Criação dos vértices do corpo da pessoa azul:
blue_p_body_start = grass_end 
blue_p_body_vertices = create_person_body_vertices()
blue_p_body_end = blue_p_body_start + len(blue_p_body_vertices)

# Criação dos vértices da cabeça da pessoa azul:
blue_p_head_start = blue_p_body_end
blue_p_head_vertices = create_circle_vertices(0.1, 30)
blue_p_head_end = blue_p_head_start + len(blue_p_head_vertices)

# Criação dos vértices do corpo da pessoa rosa:
pink_p_body_start = blue_p_head_end 
pink_p_body_vertices = create_person_body_vertices()
pink_p_body_end = pink_p_body_start + len(pink_p_body_vertices)

# Criação dos vértices da cabeça da pessoa rosa:
pink_p_head_start = pink_p_body_end
pink_p_head_vertices = create_circle_vertices(0.1, 30)
pink_p_head_end = pink_p_head_start + len(pink_p_head_vertices)

# Criação dos vértices do cabo do catavento roxo:
purple_wm_handle_start = pink_p_head_end
purple_wm_handle_vertices = create_wind_mill_handle_vertices(0.0,0.0)
purple_wm_handle_end = purple_wm_handle_start + len(purple_wm_handle_vertices)

# Criação dos vértices das pás do catavento roxo:
purple_wm_shovel_start = purple_wm_handle_end
purple_wm_shovel_vertices = create_wind_mill_shovel_vertices(0.0,0.0)
purple_wm_shovel_end = purple_wm_shovel_start + len(purple_wm_shovel_vertices)

# Criação dos vértices do cabo do catavento vermelho:
red_wm_handle_start = purple_wm_shovel_end
red_wm_handle_vertices = create_wind_mill_handle_vertices(0.0,0.0)
red_wm_handle_end = red_wm_handle_start + len(red_wm_handle_vertices)

# Criação dos vértices das pás do catavento vermelho:
red_wm_shovel_start = red_wm_handle_end
red_wm_shovel_vertices = create_wind_mill_shovel_vertices(0.0,0.0)
red_wm_shovel_end = red_wm_shovel_start + len(red_wm_shovel_vertices)

# Criação dos vértices do tronco da árvore:
tree_trunk_start = red_wm_shovel_end
tree_trunk_vertices = create_tree_trunk_vertices(-0.7, -0.6)
tree_trunk_end = tree_trunk_start + len(tree_trunk_vertices)

# Criação dos vértices da copa da árvore:
tree_top_start = tree_trunk_end
tree_top_vertices = create_tree_top_vertices(-0.8, -0.6)
tree_top_end = tree_top_start + len(tree_top_vertices)

# Criar coração
heart_start = tree_top_end
heart_vertices = create_heart_vertices()
heart_end = heart_start + len(heart_vertices)

##### Criação de Vetor Único com as Coordenadas dos Objetos

In [1367]:
# Determinação do total de vértices:
total_vertices = heart_end  # último objeto

# Preparação de espaço para os vértices, usando 2 coordenadas (x, y):
vertices = np.zeros(total_vertices, [('position', np.float32, 2)])

# Preenchimento das coordenadas do sol:
for i in range(len(sun_vertices)):
    vertices['position'][sun_start + i] = sun_vertices[i]

# Preenchimento das coordenadas da lua:
for i in range(len(moon_vertices)):
    vertices['position'][moon_start + i] = moon_vertices[i]

# Preenchimento das coordenadas da grama:
for i in range(len(grass_vertices)):
    vertices['position'][grass_start + i] = grass_vertices[i]

# Preenchimento das coordenadas do corpo da pessoa azul:
for i in range(len(blue_p_body_vertices)):
    vertices['position'][blue_p_body_start + i] = blue_p_body_vertices[i]

# Preenchimento das coordenadas da cabeça da pessoa azul:
for i in range(len(blue_p_head_vertices)):
    vertices['position'][blue_p_head_start + i] = blue_p_head_vertices[i]

# Preenchimento das coordenadas do corpo da pessoa rosa:
for i in range(len(pink_p_body_vertices)):
    vertices['position'][pink_p_body_start + i] = pink_p_body_vertices[i]

# Preenchimento das coordenadas da cabeça da pessoa rosa:
for i in range(len(pink_p_head_vertices)):
    vertices['position'][pink_p_head_start + i] = pink_p_head_vertices[i]

# Preenchimento das coordenadas do cabo do catavento roxo:
for i in range(len(purple_wm_handle_vertices)):
    vertices['position'][purple_wm_handle_start + i] = purple_wm_handle_vertices[i]

# Preenchimento das coordenadas das pás do catavento roxo:
for i in range(len(purple_wm_shovel_vertices)):
    vertices['position'][purple_wm_shovel_start + i] = purple_wm_shovel_vertices[i]

# Preenchimento das coordenadas do cabo do catavento vermelho:
for i in range(len(red_wm_handle_vertices)):
    vertices['position'][red_wm_handle_start + i] = red_wm_handle_vertices[i]

# Preenchimento das coordenadas das pás do catavento vermelho:
for i in range(len(red_wm_shovel_vertices)):
    vertices['position'][red_wm_shovel_start + i] = red_wm_shovel_vertices[i]

# Preenchimento das coordenadas do tronco da árvore:
for i in range(len(tree_trunk_vertices)):
    vertices['position'][tree_trunk_start + i] = tree_trunk_vertices[i]

# Preenchimento das coordenadas da copa da árvore:
for i in range(len(tree_top_vertices)):
    vertices['position'][tree_top_start + i] = tree_top_vertices[i]
    
# Preenchimento das coordenadas do coração
for i in range(len(heart_vertices)):
    vertices['position'][heart_start + i] = heart_vertices[i]

### Requisição de Slot de Buffer para a GPU e Envio dos Vértices

In [1368]:
# Requisição de slot de buffer da GPU:
buffer = glGenBuffers(1)

# Determinação que o buffer atual é o default:
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Envio dos dados:
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associação das Variáveis do GLSL com os Dados:

In [1369]:
# Definição do byte inicial e do offset de dados:
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

# Definição da variável position no vertex shader:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

# Indicação do conteúdo para a variável position para a GPU:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

# Obtenção da localização da variável color, para que possa ser alterada:
loc_color = glGetUniformLocation(program, "color")

### Criação de Variáveis "Globais"

In [1370]:
# Offsets de translação do sol / lua:
t_x_moon_sun = 0   # offset horizontal
t_y_moon_sun = .7  # offset vertical

# Grau de escala da árvore:
s_tree = 1

# Ângulo de rotação dos cataventos:
wm_angle = 0.0

# Flag de dia e noite:
day = True

# Escala RGB do plano de fundo:
R = 0.1
G = 0.2
B = 0.9

# Coordenadas da pessoa azul:
blue_px = 0.0 
blue_py = 0.0

# Coordenadas da pessoa rosa:
pink_px = 0.3
pink_py = 0.0

# Booleano para checar se as pessoas se encontraram:
together = False


### Definição de Constantes Associadas ao Teclado

A fim de deixar o código mais legível, associamos alguns valores numéricos de códigos de teclas a constantes de nomes correspondentes. Definimos:

In [1371]:
# Letras e seus códigos:
W_CODE = 87  # pessoa azul cima
A_CODE = 65  # pessoa azul esquerda
S_CODE = 83  # pessoa azul baixo
D_CODE = 68  # pessoa azul direita
I_CODE = 73  # pessoa rosa cima
J_CODE = 74  # pessoa rosa esquerda
K_CODE = 75  # pessoa rosa baixo
L_CODE = 76  # pessoa rosa direita

# Setas e seus códigos:
RIGHT_CODE = 262  # rotação dos cataventos direção 1
LEFT_CODE  = 263  # rotação dos cataventos direção 2
DOWN_CODE  = 264  # escala vertical menor
UP_CODE    = 265  # escala vertical maior

### Definição de Constantes Associadas à Coordenadas

In [1372]:
# Limites das posições em que as pessoas podem ocupar:
PERSON_MAX_X = +0.8
PERSON_MIN_X = -0.8
PERSON_MAX_Y = +0.6
PERSON_MIN_Y = -0.55

### Função para Capturar Eventos de Teclado


In [1373]:
def key_event(window, key, scancode, action, mods):
    global blue_px, blue_py, pink_px, pink_py, s_tree, wm_angle
    # # Atualização do px da pessoa azul:
    if key == D_CODE and blue_px <= PERSON_MAX_X - 0.01: blue_px += 0.01  # direita
    if key == A_CODE and blue_px >= PERSON_MIN_X + 0.01: blue_px -= 0.01  # esquerda

    # # Atualização do py da pessoa azul:
    if key == W_CODE and blue_py <= PERSON_MAX_Y - 0.01: blue_py += 0.01  # cima
    if key == S_CODE and blue_py >= PERSON_MIN_Y + 0.01: blue_py -= 0.01  # baixo

    # # Atualização do px da pessoa rosa:
    if key == L_CODE and pink_px <= PERSON_MAX_X - 0.01: pink_px += 0.01  # direita
    if key == J_CODE and pink_px >= PERSON_MIN_X + 0.01: pink_px -= 0.01  # esquerda

    # # Atualização do py da pessoa rosa:
    if key == I_CODE and pink_py <= PERSON_MAX_Y - 0.01: pink_py += 0.01  # cima
    if key == K_CODE and pink_py >= PERSON_MIN_Y + 0.01: pink_py -= 0.01  # baixo

    # Atualização do parâmetro de escala da árvore:
    if key == UP_CODE and s_tree < 2.0: s_tree += 0.01    # aumentar escala
    if key == DOWN_CODE and s_tree > 0.5: s_tree -= 0.01  # diminuir escala

    # Atualização do ângulo de rotação dos cataventos:
    if key == LEFT_CODE: wm_angle += 0.02   # rotação anti-horária
    if key == RIGHT_CODE: wm_angle -= 0.02  # rotação horária

### Função que determina se as 2 pessoas colidiram

In [1374]:
def are_people_colliding():
    global blue_px, blue_py, pink_px, pink_py

    dist = np.sqrt( (blue_px - pink_px) ** 2 + (blue_py - pink_py ) ** 2)

    return dist < 0.2

### Função que multiplica 2 Matrizes

In [1375]:
def multiply_matrices(a, b):
    m_a = a.reshape(4, 4)
    m_b = b.reshape(4, 4)
    m_c = np.dot(m_a, m_b)
    c = m_c.reshape(1, 16)
    return c

### Função que cria uma Matriz de Translação

In [1376]:
def create_translation_matrix(t_x, t_y):
    translation_matrix = np.array([ 1.0, 0.0, 0.0, t_x, 
                                    0.0, 1.0, 0.0, t_y, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    return translation_matrix

### Função que cria uma Matriz de Rotação

In [1377]:
def create_rotation_matrix(angle):
    rotation_matrix = np.array([ np.cos(angle), -np.sin(angle), 0.0, 0.0, 
                                 np.sin(angle),  np.cos(angle), 0.0, 0.0, 
                                 0.0,            0.0,           1.0, 0.0, 
                                 0.0,            0.0,           0.0, 1.0], np.float32)
    return rotation_matrix

### Função que cria uma Matriz de Escala

In [1378]:
def create_scale_matrix(s, x_f=0, y_f=0):
    scale_matrix = np.array([ s,   0.0, x_f * (1 - s), 0.0, 
                              0.0, s,   y_f * (1 - s), 0.0, 
                              0.0, 0.0, 1.0, 0.0, 
                              0.0, 0.0, 0.0, 1.0], np.float32)
    return scale_matrix

### Funçao que cria uma Matriz Identidade

In [1379]:
def create_identity_matrix():
    no_transformation_matrix = np.array([ 1.0, 0.0, 0.0, 0.0, 
                                          0.0, 1.0, 0.0, 0.0, 
                                          0.0, 0.0, 1.0, 0.0, 
                                          0.0, 0.0, 0.0, 1.0],
                                          np.float32)
    return no_transformation_matrix

### Função para Desenho da Cor de Fundo Diurna

In [1380]:
def draw_day_time_background():
    global t_x_moon_sun, R, G, B

    # Atualização da cor:
    if t_x_moon_sun < 0:
        B += 0.000355
    else:
        B -= 0.000355
    
    # Limpeza do buffer e sobreposição com a cor atualizada:
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(R, G, B, 1.0)

### Função para Desenho da Cor de Fundo Noturna

In [1381]:
def draw_night_time_background():
    global R, G, B

    # Limpeza do buffer e sobreposição com a cor estática:
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(R, G, B, 1.0)

### Função para Desenho de Cor de Fundo Aleatória

In [1382]:
def draw_random_color_background():
    # Limpeza do buffer e sobreposição com a cor aleatória:
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(1, .7, .75, 1)

### Função para Desenho do Sol

In [1383]:
def draw_sun():
    global loc, loc_color, t_x_moon_sun, t_y_moon_sun, day, R, G, B

    # Definição da cor do sol:
    glUniform4f(loc_color, 1, .8, .1, 1.0)  # amarelo

    # Atualização da posição:
    t_x_moon_sun += 0.00071
    glUniformMatrix4fv(loc, 1, GL_TRUE, create_translation_matrix(t_x_moon_sun, t_y_moon_sun))

    # Desenho dos vértices:
    glDrawArrays(GL_TRIANGLE_FAN, sun_start, len(sun_vertices))

    # Verificação da troca de período:
    if t_x_moon_sun >= 1.2:
        day = False
        t_x_moon_sun = -1.2
        R, G, B = .1, .1, .1

### Função para Desenho da Lua

In [1384]:
def draw_moon():
    global loc, loc_color, t_x_moon_sun, t_y_moon_sun, day, R, G, B

    # Definição da cor da lua:
    glUniform4f(loc_color, .9, .9, .95, 1.0)  # cinza azulado

    # Atualização da posição:
    t_x_moon_sun += 0.0005
    glUniformMatrix4fv(loc, 1, GL_TRUE, create_translation_matrix(t_x_moon_sun, t_y_moon_sun))

    # Desenho dos vértices:
    glDrawArrays(GL_TRIANGLE_FAN, moon_start, len(moon_vertices))

    # Verificação da troca de período:
    if t_x_moon_sun >= 1.2:
        day = True
        t_x_moon_sun = -1.2
        R, G, B = .1, .2, .3

### Função para Desenho da Grama

In [1385]:
def draw_grass():
    global loc, loc_color

    # Definição da cor da grama:
    glUniform4f(loc_color, .3, 1, .3, 1.0)  # verde claro

    # Desenho dos vértices (posição estática):
    glUniformMatrix4fv(loc, 1, GL_TRUE, create_identity_matrix())
    glDrawArrays(GL_TRIANGLES, grass_start, len(grass_vertices))

### Função para Desenho da Pessoa Azul

In [1386]:
def draw_blue_person():
    global blue_px, blue_py
    
    # Definição da cor:
    glUniform4f(loc_color, 0, 0.5, 1, 1.0)  # cor azul

    # Desenho dos vértices do corpo:
    glUniformMatrix4fv(loc, 1, GL_TRUE, create_translation_matrix(blue_px, blue_py))
    glLineWidth(4.0)
    glDrawArrays(GL_LINE_STRIP, blue_p_body_start, len(blue_p_body_vertices))

    # Desenho dos vértices da cabeça:
    glDrawArrays(GL_TRIANGLE_FAN, blue_p_head_start, len(blue_p_head_vertices))  


### Função para Desenho da Pessoa Rosa

In [1387]:
def draw_pink_person():
    global pink_px, pink_py
    
    # Definição da cor:
    glUniform4f(loc_color, 1, 0.2, .9, 1.0)  # cor rosa

    # Desenho dos vértices do corpo:
    glUniformMatrix4fv(loc, 1, GL_TRUE, create_translation_matrix(pink_px, pink_py))
    glLineWidth(4.0)
    glDrawArrays(GL_LINE_STRIP, pink_p_body_start, len( pink_p_body_vertices))

    # Desenho dos vértices da cabeça:
    glDrawArrays(GL_TRIANGLE_FAN,  pink_p_head_start, len( pink_p_head_vertices))  

### Função para Desenho do Catavento Roxo

In [1388]:
def draw_purple_wind_mill():
    
    global blue_px, blue_py, wm_angle
    
    # Definição da cor do cabo:
    glUniform4f(loc_color, 0, 128/255, 0/255, 1.0)  # cor verde
    
    # Desenho do cabo do catavento:
    translation_matrix = create_translation_matrix(blue_px + 0.08, blue_py + 0.1)
    glUniformMatrix4fv(loc, 1, GL_TRUE, translation_matrix)
    glDrawArrays(GL_LINE_STRIP, purple_wm_handle_start, len(purple_wm_handle_vertices))

    # Definição da cor das pás:
    glUniform4f(loc_color, 150/255, 0, 205/255.0, 1.0)  # cor roxa

    # Desenho das pás do catavento:
    multiplication_matrix = multiply_matrices(create_rotation_matrix(wm_angle), create_scale_matrix(1))
    final_matrix = multiply_matrices(translation_matrix, multiplication_matrix)
    glUniformMatrix4fv(loc, 1, GL_TRUE, final_matrix)
    glDrawArrays(GL_TRIANGLE_FAN, purple_wm_shovel_start, len(purple_wm_shovel_vertices))

### Função para Desenho do Catavento Vermelho

In [1389]:
def draw_red_wind_mill():
    
    global pink_px, pink_py, wm_angle
    
    # Definição da cor do cabo:
    glUniform4f(loc_color, 0, 228/255, 0/255, 1.0)  # cor verde

    # Desenho do cabo do catavento:
    translation_matrix = create_translation_matrix(pink_px - 0.08, pink_py + 0.1)
    glUniformMatrix4fv(loc, 1, GL_TRUE, translation_matrix)
    glDrawArrays(GL_LINE_STRIP, red_wm_handle_start, len(red_wm_handle_vertices))

    # Definição da cor das pás:
    glUniform4f(loc_color, 250/255, 0, 150/255, 1.0)  # cor vermelha arrosada

    # Desenho das pás do catavento:
    multiplication_matrix = multiply_matrices(create_rotation_matrix(wm_angle), create_scale_matrix(1))
    final_matrix = multiply_matrices(translation_matrix, multiplication_matrix)
    glUniformMatrix4fv(loc, 1, GL_TRUE, final_matrix)
    glDrawArrays(GL_TRIANGLE_FAN, red_wm_shovel_start, len(red_wm_shovel_vertices))

### Função para Desenho da Árvore

In [1390]:
def draw_tree():
    global loc, loc_color, s_tree

    # Definição da cor do tronco:
    glUniform4f(loc_color, .6, .3, 0, 1.0)  # marrom

    # Desenho dos vértices do tronco:
    glUniformMatrix4fv(loc, 1, GL_TRUE, create_scale_matrix(s_tree))
    glDrawArrays(GL_TRIANGLE_FAN, tree_trunk_start, len(tree_trunk_vertices))

    # Definição da cor da árvore:
    glUniform4f(loc_color, .2, .5, .2, 1.0)  # verde escuro

    # Desenho dos vértices da copa:
    # glUniformMatrix4fv(loc, 1, GL_TRUE, create_scale_matrix(s_tree))
    glDrawArrays(GL_TRIANGLE_FAN, tree_top_start, len(tree_top_vertices))

### Função para Desenho do Coração

In [1391]:
def draw_heart():
    global loc, loc_color, blue_px, blue_py
    
    # Definição da cor do coração:
    glUniform4f(loc_color, 1, 0, 0, 1.0)  # cor vermelha

    # Desenho dos vértices (a partir da posição da pessoa azul):
    translation_matrix = create_translation_matrix(blue_px - 1.3, blue_py - 0.8)
    multiplication_matrix = multiply_matrices(translation_matrix, create_scale_matrix(7))
    glUniformMatrix4fv(loc, 1, GL_TRUE, multiplication_matrix)    
    glLineWidth(4.0)
    glDrawArrays(GL_TRIANGLE_FAN, heart_start, len(heart_vertices))

### Exibição da Janela

In [1392]:
glfw.show_window(window)
glfw.set_key_callback(window,key_event)

### Loop Principal da Janela

Enquanto a janela não for fechada, executamos o nosso laço principal:

In [ ]:
while not glfw.window_should_close(window):
    global together

    # Captação de eventos glfw:
    glfw.poll_events() 

    # Desenho do cenário diurno ou noturno:
    if day:
        if not together:
            draw_day_time_background()
        else:
            draw_random_color_background()
            draw_heart()
        draw_sun()
    else:
        if not together:
            draw_night_time_background()
        else:
            draw_random_color_background()
            draw_heart()
        draw_moon()
    
    # Desenho da árvore:
    draw_tree()

    # Desenho da grama:
    draw_grass()

    # Desenho dos cataventos:
    draw_purple_wind_mill()
    draw_red_wind_mill()

    # Desenho das pessoas:
    draw_blue_person()
    draw_pink_person()

    # Atualização do controle de colisão das pessoas:
    together = are_people_colliding()

    # Atualização da localização de mat_transformation:
    loc = glGetUniformLocation(program, 'mat_transformation')

    # Troca de buffers glfw:
    glfw.swap_buffers(window)


### Finalização da Execução

Quando a janela for fechada, o laço principal será interrompido e podemos finalizar o sistema de janela _GLFW_:

In [ ]:
glfw.terminate()